In [51]:
import sys
import os


sys.path.append(os.path.abspath(".."))


In [52]:
os.path.abspath("..")


'C:\\Users\\prani\\Documents\\stock_strategy_analyzer'

In [53]:
from src.strategies import rsi_signal, ma_signal, bb_signal

In [54]:
import pandas as pd

feature_data = pd.read_csv("../features/market_features_with_regime.csv")
feature_data["Date"] = pd.to_datetime(feature_data["Date"])
feature_data = feature_data.set_index("Date")

feature_data.head()

,returns,volatility_20,trend_strength,range_20,regime,Close
Date,,,,,,
2015-03-17,0.010442,0.009467,0.014991,0.012390,0,8723.299805
2015-03-18,-0.004287,0.009503,0.013599,0.012434,0,8685.900391
2015-03-19,-0.005900,0.009403,0.011650,0.012957,0,8634.650391
2015-03-20,-0.007383,0.009441,0.008751,0.012725,1,8570.900391
2015-03-23,-0.002333,0.009365,0.006085,0.012648,1,8550.900391


In [55]:
price_data = pd.read_csv("../data/nifty50.csv", header=[0,1], index_col=0)
price_data.columns = price_data.columns.get_level_values(0)
price_data.index = pd.to_datetime(price_data.index, errors="coerce")
price_data = price_data.dropna()


In [56]:
data = price_data.loc[feature_data.index].copy()
data["regime"] = feature_data["regime"]


In [57]:
def compute_rsi(series, window=14):
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_gain = gain.rolling(window).mean()
    avg_loss = loss.rolling(window).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi


In [58]:
data["RSI"] = compute_rsi(data["Close"])


In [59]:
data["rsi_signal"] = data["RSI"].apply(rsi_signal)



In [60]:
data["strategy_return"] = data["rsi_signal"].shift(1) * data["Close"].pct_change()



In [61]:
performance = data.groupby("regime")["strategy_return"].agg([
    "mean",
    "std",
    "count"
])

performance["sharpe"] = performance["mean"] / performance["std"]
performance


,mean,std,count,sharpe
regime,,,,
0,-0.000204,0.003730,1756,-0.054658
1,0.000057,0.007453,871,0.007614
2,0.000672,0.034284,30,0.019613


In [62]:
data["MA_20"] = data["Close"].rolling(20).mean()
data["MA_50"] = data["Close"].rolling(50).mean()


In [63]:
data["ma_signal"] = data.apply(
    lambda r: ma_signal(r["MA_20"], r["MA_50"]),
    axis=1
)



In [64]:
data["ma_strategy_return"] = (
    data["ma_signal"].shift(1) * data["Close"].pct_change()
)


In [65]:
ma_performance = data.groupby("regime")["ma_strategy_return"].agg([
    "mean",
    "std",
    "count"
])

ma_performance["sharpe"] = ma_performance["mean"] / ma_performance["std"]
ma_performance


,mean,std,count,sharpe
regime,,,,
0,0.000478,0.007238,1756,0.066100
1,0.000475,0.012108,871,0.039238
2,-0.001785,0.042911,30,-0.041596


In [66]:
data["BB_MID"] = data["Close"].rolling(20).mean()
data["BB_STD"] = data["Close"].rolling(20).std()

data["BB_UPPER"] = data["BB_MID"] + 2 * data["BB_STD"]
data["BB_LOWER"] = data["BB_MID"] - 2 * data["BB_STD"]


In [67]:
data["bb_signal"] = data.apply(
    lambda r: bb_signal(r["Close"], r["BB_LOWER"], r["BB_UPPER"]),
    axis=1
)



In [68]:
data["bb_strategy_return"] = (
    data["bb_signal"].shift(1) * data["Close"].pct_change()
)


In [69]:
bb_performance = data.groupby("regime")["bb_strategy_return"].agg([
    "mean",
    "std",
    "count"
])

bb_performance["sharpe"] = bb_performance["mean"] / bb_performance["std"]
bb_performance


,mean,std,count,sharpe
regime,,,,
0,-0.000091,0.002034,1756,-0.044876
1,0.000028,0.005406,871,0.005219
2,-0.001825,0.012040,30,-0.151550


In [71]:
data["adaptive_signal"] = 0.0

data.loc[data["regime"] == 0, "adaptive_signal"] = (
    data.loc[data["regime"] == 0, "ma_signal"]
)

data.loc[data["regime"] == 2, "adaptive_signal"] = (
    data.loc[data["regime"] == 2, "rsi_signal"]
)


data.loc[data["regime"] == 1, "adaptive_signal"] = (
    data.loc[data["regime"] == 1, ["rsi_signal", "ma_signal", "bb_signal"]]
    .mean(axis=1)
)




In [72]:
data["adaptive_return"] = (
    data["adaptive_signal"].shift(1) * data["Close"].pct_change()
)


In [73]:
adaptive_perf = data["adaptive_return"].agg(["mean", "std", "count"])
adaptive_perf["sharpe"] = adaptive_perf["mean"] / adaptive_perf["std"]
adaptive_perf


mean         0.000337
std          0.007526
count     2657.000000
sharpe       0.044739
Name: adaptive_return, dtype: float64

In [74]:
comparison = pd.DataFrame({
    "RSI": data["strategy_return"],
    "MA": data["ma_strategy_return"],
    "Bollinger": data["bb_strategy_return"],
    "Adaptive": data["adaptive_return"]
})

(comparison.mean() / comparison.std())


RSI         -0.017109
MA           0.044559
Bollinger   -0.019206
Adaptive     0.044739
dtype: float64

In [75]:
data.groupby("regime")["adaptive_return"].agg([
    "mean", "std", "count"
]).assign(
    sharpe=lambda x: x["mean"] / x["std"]
)


,mean,std,count,sharpe
regime,,,,
0,0.000473,0.007194,1756,0.065796
1,0.000007,0.005688,871,0.001246
2,0.001907,0.032825,30,0.058102


In [76]:
data[["rsi_signal", "ma_signal", "bb_signal"]].head()
data[["rsi_signal", "ma_signal", "bb_signal"]].describe()


Price,rsi_signal,ma_signal,bb_signal
count,2658.000000,2658.000000,2658.000000
mean,-0.145598,0.366817,-0.009782
std,0.510578,0.920505,0.295332
min,-1.000000,-1.000000,-1.000000
25%,0.000000,-1.000000,0.000000
50%,0.000000,1.000000,0.000000
75%,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000
